In [12]:
import os
import requests
from dotenv import load_dotenv

load_dotenv(".env")

API_URL = "https://openrouter.ai/api/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {os.environ["OPENROUTER_API_KEY"]}",
    "Content-Type": "application/json",
}

RQ = """Task: Identify and correct code smells in the provided **Python** code snippet. If there is no discernible code smell, respond with 'None'. Then, suggest which modifications or refactorings could be applied to mitigate the identified code smell.

Instructions:  
1. Analyze the code for code smells (e.g., duplicated code, long methods, magic numbers, poor naming).  
2. For each smell, provide:  
   - **Type**: A tuple `(code_smell_type, start_line, end_line)`.  
   - **Correction**: The **entire rewritten code** with all fixes applied, marking new sections with `====== [ADDED CODE] =======`.  

3. **Output Format**:  

Code Smells Detected:
Type: (CodeSmellType, start_line, end_line)
Type: (CodeSmellType, start_line, end_line)
...

Corrected Code
====== [CORRECTED CODE START] =======
[Full corrected code here]
====== [CORRECTED CODE END] =======
"""

In [ ]:
def make_question(request, snippet):
    return f"""{request}

{snippet}"""

def load_snippets(folder_path):
    snippets = []
    for root, _, files in os.walk(folder_path):
        py_files = sorted(f for f in files if f.endswith(".py"))
        for filename in py_files:
            file_path = os.path.join(root, filename)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                    snippets.append((filename, content))
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
    return snippets


def save_markdown(filename, content):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    try:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(content)
        print(f"Successfully saved content to '{filename}'")
    except Exception as e:
        print(f"Error saving to file: {e}")

In [ ]:
def process_snippets(request=RQ, snippets=[]):
    """Process code snippets with the given request."""
    print(snippets)
    for filename, snippet in snippets:
        try:
            data = {
                "model": "deepseek/deepseek-chat:free",
                "messages": [{"role": "user", "content": make_question(request, snippet)}],
            }

            response = requests.post(API_URL, json=data, headers=headers)
            print(f"Response status code: {response.status_code}")
            if response.status_code == 200:
                parsed_response = response.json()
                answer = """"""
                for index, resp in enumerate(parsed_response["choices"]):
                    content = resp["message"]["content"]
                    if index == 0:
                        answer += content
                    else:
                        answer += "\n\n" + content
                save_markdown(f"responses/{filename}.md", answer)
            else:
                print(
                    "Falha ao buscar dados da API do OpenRouter. Código de Status:",
                    response.status_code,
                )
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")

In [21]:
snippets_folder = "../pyparsing/pyparsing"
snippets = load_snippets(snippets_folder)
request = RQ

process_snippets(request=request, snippets=snippets)

[('__init__.py', '# module pyparsing.py\n#\n# Copyright (c) 2003-2022  Paul T. McGuire\n#\n# Permission is hereby granted, free of charge, to any person obtaining\n# a copy of this software and associated documentation files (the\n# "Software"), to deal in the Software without restriction, including\n# without limitation the rights to use, copy, modify, merge, publish,\n# distribute, sublicense, and/or sell copies of the Software, and to\n# permit persons to whom the Software is furnished to do so, subject to\n# the following conditions:\n#\n# The above copyright notice and this permission notice shall be\n# included in all copies or substantial portions of the Software.\n#\n# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,\n# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF\n# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.\n# IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY\n# CLAIM, DAMAGES OR OTHER LIABILI